In [1]:
# import some libraries we'll need
import numpy as np
from matplotlib import pyplot as plt
import torch

Suppose we have a neural network with two inputs, two hidden layers of 8 units each, ReLU activations (except for $\hat y = f_\text{out}(h_L)$) and one output.

* Write an algorithm using python to compute the NN output $\hat y$ given the input and the free parameters (weights and biases)
* Write an algorithm to compute derivatives of the least squares loss $(y - \hat y)^2$ with respect to all free parameters of the NN. This should be a python function that takes the free parameters, NN inputs $x$ and NN training targets $y$ as its own inputs. **Don't use automatic differntiation, finite differences or symbolic math tools: do the derivatives by hand!**
* **Optional challenge problem (+25% of assignment max. points), skip for now if you get stuck**: Extend your derivative computation algorithm to work with any number of hidden layers, all of which can have different numbers of hidden units.

In [ ]:
import numpy as np

def initialize_weights_and_biases(layer_sizes):
    weights = []
    biases = []
    for i in range(len(layer_sizes) - 1):
        weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01)
        biases.append(np.zeros(layer_sizes[i + 1]))
    return weights, biases

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def forward_pass(x, y, weights, biases):
    layer = len(weights)-1
    h = x
    listh = [h]
    for i in range(layer):
        W,b = weights[i+1], biases[i+1]
        H = h@ W +b
        if i < layer -1:
            H = relu(H)
            listh.append(H)
            h = H
        else:
            y_hat = H
    
    return y_hat, listh

def backward_propagation(x, y, y_hat, listh, weights, biases):
    # Backward pass
    loss =y - y_hat
    layer=len(weights)-2
    h = listh
    listdWl = [], listdW =[]
    listdbl = [], listdb =[]
    listdhl = [], listdh =[]
    de = -2 * (loss).T
    dhL=weights[-1]
    de_dhL=de.dot(dhL.T)
    de_dbL = de_dhL.dot(relu_derivative(weights[layer].dot(h[layer-1])+biases[layer]))
    de_dWl = de_dbL.dot(h[layer-1]).T
    listdbl.append(de_dbL)
    listdWl.append(de_dWl)
    listdhl.append(de_dhL)
    dhj= dhL
    for i in range(layer, -1, -1):
        de_dhj= de.dot(dhj)*relu_derivative(weights[i+1].dot(h[i])+biases[i+1])*weights[i+1]
        de_dbj=de_dhj*relu_derivative(weights[i].dot(h[i-1])+biases[i])
        de_dwj=de_dhj*relu_derivative(weights[i].dot(h[i-1])+biases[i])*h[i-1].T
        listdb.append(de_dbj)
        listdW.append(de_dwj)
        listdh.append(de_dhj)
        dhj = de_dhj
    return listdW, listdb, listdh, listdWl, listdbl, listdhl

In [8]:
layer_sizes = [2, 8, 8, 1]
x = np.random.randn(1, 2)
y = np.zeros((1, 1))
# Initialize the weights and biases
weights, biases = initialize_weights_and_biases(layer_sizes)
y_hat, listh=forward_pass(x, y, weights, biases)
listdW, listdb, listdh, listdWl, listdbl, listdhl=backward_propagation(x, y, y_hat, listh, weights, biases)
#print('Input  x     :',x)
#print('Gradient of weights:', grad_weights)
#print('Gradient of biases:', grad_biases)
#print('Output y_hat:', forward_pass(x, weights, biases))
print(grad_weights[0].shape)
print(grad_biases[0].shape)


ValueError: operands could not be broadcast together with shapes (1,2) (8,8) 

We'll now download a data file and extract 3 variables. Each is a NumPy array:
* 'y' contains monthly values of the [Nino 3.4 ENSO index](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni), describing the [El Nino Southern Oscillation](https://en.wikipedia.org/wiki/El_Ni%C3%B1o%E2%80%93Southern_Oscillation) over time.
* `x` contains empirical orthogonal function coefficients that describe the ocean temperature at the sea surface and the depth average over the top 300 meters, in the Indo-Pacific, North-Pacific and Atlantic regions. Essentially, the numbers in each row of $x$ summarize temperatures in the upper ocean across the globe. For details of how these are calculated, you can consult [this paper](https://www.nature.com/articles/s41586-019-1559-7) by Ham et al. Each row of `x` describes the ocean state 2 months before the corresponding element of `y`.
* `t` contains the time in months since Jan. 1 1970 for each value of `y`. We can use this for plotting results but it won't appear in our calculations otherwise.

In [11]:
# library for downloading files from google drive.
!pip install gdown

# download the data
!gdown https://drive.google.com/uc?id=1FUb-2lcAd0Y1ULjx5jB6UTMNDmGy3vZA

Downloading...
From: https://drive.google.com/uc?id=1FUb-2lcAd0Y1ULjx5jB6UTMNDmGy3vZA
To: c:\Users\papas\Documents\GitHub\Machine-Learning-2nd-semester\enso_and_pca.npz

  0%|          | 0.00/242k [00:00<?, ?B/s]
100%|██████████| 242k/242k [00:00<00:00, 1.56MB/s]
100%|██████████| 242k/242k [00:00<00:00, 1.18MB/s]


Normalize `y` and each column of `x` so they all have a mean of zero and a standard deviation of one. Overwrite the original variables `x,y` with the normalized ones, but keep track of what you've done, so you can convert your predictions back into original data units later on if needed.

In [12]:
# load the data
with np.load('enso_and_pca.npz') as data:
    t, x, y = data['t'], data['x'], data['y']

mx = x.mean(axis=0)
sx = x.std(axis=0)
my = y.mean(axis=0)
sy = y.mean(axis=0)
x = (x - x.mean(axis=0))/x.std(axis=0)
y = (y - y.mean())/y.std()

Last time we wrote a loop by hand to cycle over the data. However, pytorch provides great utilities to do this for us, which will let us focus on what's new in each lesson instead. In particular, we're going to use the PyTorch Dataset and DataLoader classes.

In [13]:
from torch.utils.data import TensorDataset, DataLoader

# create PyTorch tensors for our inputs and target outputs
xt = torch.tensor(x)
yt = torch.tensor(y)

# use the first 1100 time points as training data
xt_train, yt_train = xt[:1100], yt[:1100]

# save the remaining data for testing
xt_test, yt_test = xt[1100:], yt[1100:]

dataset = TensorDataset(xt_train, yt_train)  # combine the inputs and outputs into a PyTorch Dataset object
# create a dataloader to serve up batches of 8 data point for training
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

ModuleNotFoundError: No module named 'torch'

Now let's get the first 'batch' of data from the dataloader, and see what it looks like.

In [ ]:
for x_batch, y_batch in data_loader:
    print('shape of x_batch: {0}'.format(x_batch.shape) )
    print('shape of y_batch: {0}'.format(y_batch.shape) )
    break  # if we didn't have this line, the for loop would cycle through all the data

Now use a for loop, iterating over batches from the data loader, to fit a linear regression $\hat y = x\cdot \beta$ with least squares loss, by stochastic gradient descent. Refer back to the code you wrote from the last set of exercises for guidance (and feel free to copy-paste from your own previous homework).
* Add an extra column of ones to `x` to incorporate a constant term
* Don't forget to set `requires_grad = True` when initializing `beta`
* Choose how to initialize $\beta$. All zeros? Random? Does it make much of a difference here?
* As before, don't forget to use `with torch.no_grad():` when updating parameters
* Store the loss at each iteration of the loop.
* Does the loss decrease on the training data? Do you get a positive correlation between $y$ and $\hat y$ at the end on the training data?
* See if you can get a better result by adjusting the initialization of free parameters or the learning rate

Now repeat the process, but instead of a linear model we'll use a neural network with two hidden layers of 32 units each, and the ReLU activation function $\phi(z) = \max(0, z)$. Again we'll use least squares loss and a batch size of 8.

**Don't** use PyTorch's built-in classes for this just yet. Instead:
* Define tensors of the correct size and data type (check `x.dtype`) for each variable containing free parameters (weights and biases) of your neural network. Remember to set `requires_grad = True` where needed.
* For each batch of data, compute the hidden state activations $h_1$ as a function of $W_1, b_1, x$
* Then compute $h_2$, $\hat y$ and $e=\ell(y, \hat y)$.
* Call `e.backward()` to compute derivatives of $e$ with respect to the free variables of your neural network, using backward-mode differentiation.
* Now update all free parameters based on the computed derivatives and the learning rate, just as you did for the linear regression example. Remember to use `with torch.no_grad():`
* Plot the convergence of the loss function over iterations. Are you getting a better fit than with the linear regression?
* How did you initialize? Does it make a difference now?

Now plot the target ENSO 3.4 index values $y$ against the predictions from the linear regression and the neural network. Clearly mark the boundary between training and testing data.

What are mean square errors for the NN and linear regression, in original data units, on the testing and training data? How about Pearson's correlation coefficient?

Now we'll do the same thing, but defining a python object class for our neural network, and some built-in PyTorch classes:

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_inputs=46, n_outputs=1, n_hidden=32):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(n_inputs, n_hidden)  # 5*5 from image dimension
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_outputs)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

As described in the first two lectures, we are now building our neural net out of composable functions that we can stack together like legos.

PyTorch also lets us iterate in loops over the trainable parameters of the model. Note how `requires_grad` was correctly set by default for the learnable parameters.

In [ ]:
net = Net()
for name, param in net.named_parameters():
    print(f'{name}: requires_grad={param.requires_grad}')

When we use our network object like a function, the input gets passed to the `forward()` method:

In [ ]:
print(net(torch.rand(46)))

We can also provide it with multiple inputs to process independently:

In [ ]:
print(net(torch.rand(4,46)))

Now redo the training loop from before, but now update the parameters using an inner loop over `net.named_parameters` (remember to use `torch.no_grad()` when updating parameters, and to zero out the derivatives after you do so).

Instead of calling `var.grad.zero()` on each trainable variable, we can now also simply call `net.zero_grad()` once.

Look up the default initialization of weights and biases in the `nn.Linear` objects you used.

Change the `Net()` class above to allow an adjustable number of hidden layers to be specified when initializing the object, and to specify a different number of hidden units for each hidden layer. Can you get better results on the testing data by adjusting this?